In [19]:
import os
import cv2
import numpy as np
import random
from xml.etree.ElementTree import Element, SubElement, tostring, ElementTree
import time

In [11]:
def add_watermark(image_path, watermark_path, output_image_path):
    image = cv2.imread(image_path)
    watermark = cv2.imread(watermark_path, cv2.IMREAD_UNCHANGED)

    h, w, _ = image.shape
    wh, ww, _ = watermark.shape

    if wh > h or ww > w:
        raise ValueError('Watermark dimensions are larger than the input image dimensions.')

    x = random.randint(0, w - ww)
    y = random.randint(0, h - wh)

    overlay = np.zeros_like(image)
    overlay[y:y+wh, x:x+ww] = watermark[:, :, 0:3]

    alpha = watermark[:, :, 3] / 255.0
    alpha = alpha.astype(np.float32)    
    alpha_expanded = np.expand_dims(alpha, axis=2)
    alpha_overlay = np.repeat(alpha_expanded, repeats=3, axis=2).astype(np.double) # convert alpha_overlay to type double
    alpha_overlay = alpha_overlay / np.max(alpha_overlay)
    image[y:y+wh, x:x+ww] = cv2.addWeighted(image[y:y+wh, x:x+ww], 1, overlay[y:y+wh, x:x+ww], 0.5, gamma=0.5)


    os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
    cv2.imwrite(output_image_path, image)
    return x, y, ww, wh


In [18]:
def create_annotation(image_path, output_annotation_path, bbox):
    x, y, width, height = bbox
    
    image = cv2.imread(image_path)
    h, w, _ = image.shape

    root = Element('annotation')


    filename = SubElement(root, 'filename')
    filename.text = os.path.basename(image_path)

    size = SubElement(root, 'size')
    width_element = SubElement(size, 'width')
    width_element.text = str(w)
    height_element = SubElement(size, 'height')
    height_element.text = str(h)
    depth_element = SubElement(size, 'depth')
    depth_element.text = '3'

    segmented = SubElement(root, 'segmented')
    segmented.text = '0'

    object_element = SubElement(root, 'object')
    name = SubElement(object_element, 'name')
    name.text = 'watermark'
    pose = SubElement(object_element, 'pose')
    pose.text = 'Unspecified'
    truncated = SubElement(object_element, 'truncated')
    truncated.text = '0'
    difficult = SubElement(object_element, 'difficult')
    difficult.text = '0'

    bndbox = SubElement(object_element, 'bndbox')
    xmin = SubElement(bndbox, 'xmin')
    xmin.text = str(x)
    ymin = SubElement(bndbox, 'ymin')
    ymin.text = str(y)
    xmax = SubElement(bndbox, 'xmax')
    xmax.text = str(x + width)
    ymax = SubElement(bndbox, 'ymax')
    ymax.text = str(y + height)

    tree = ElementTree(root)
    tree.write(output_annotation_path)



In [25]:
IMAGE_PATH = 'train/no-watermark'
WATERMARK_PATH = 'watermark.png'
OUTPUT_TRAIN_IMAGE_PATH = 'TensorFlow/workspace/training_demo/images/train'

OUTPUT_TEST_IMAGE_PATH = 'TensorFlow/workspace/training_demo/images/test'

OUTPUT_TRAIN_ANNOTATION_PATH = 'TensorFlow/workspace/training_demo/images/train'
OUTPUT_TEST_ANNOTATION_PATH = 'TensorFlow/workspace/training_demo/images/test'

In [26]:
#create files with watermark
start_time = time.time()
i = 1
for im in os.listdir(IMAGE_PATH):
  if im.endswith('.jpg') or im.endswith('.jpeg') or im.endswith('.png'):
    image_path = os.path.join(IMAGE_PATH, im)
    filename, ext = os.path.splitext(os.path.basename(image_path))
    new_filename = filename + 'w' + ext
    if(i<=10000):
        output_path = os.path.join(OUTPUT_TRAIN_IMAGE_PATH, new_filename)
    else:
        output_path = os.path.join(OUTPUT_TEST_IMAGE_PATH, new_filename)
    bbox = add_watermark(image_path, WATERMARK_PATH, output_path)

    annote_filename = filename + 'w.xml'
    if(i<=10000):
        output_annotation_path = os.path.join(OUTPUT_TRAIN_ANNOTATION_PATH, annote_filename)
    else:
        output_annotation_path = os.path.join(OUTPUT_TEST_ANNOTATION_PATH, annote_filename)
    
    if not os.path.exists(os.path.dirname(output_annotation_path)):
        os.makedirs(os.path.dirname(output_annotation_path))
    create_annotation(output_path, output_annotation_path, bbox)
    i += 1
    
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

Elapsed time: 63.660650968551636 seconds


In [ ]:
import shutil

for im in os.listdir(IMAGE_PATH):
  if im.endswith('w.jpeg'):
    shutil.move(os.path.join(IMAGE_PATH,im), OUTPUT_IMAGE_PATH)

Once the images are separated into ones without watermarks (Imagesnw) and ones with watermarks (Imagesw)